##  **1. 필요한 라이브러리 불러오기**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #for plotting
from sklearn.ensemble import RandomForestClassifier #for the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz #plot tree
from sklearn.metrics import * #for model evaluation
from sklearn.model_selection import train_test_split #for data splitting
import eli5 #for purmutation importance
from eli5.sklearn import PermutationImportance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import RandomForestRegressor
import shap #for SHAP values
shap.initjs()
import sklearn.inspection
import lime
from lime import lime_tabular
np.random.seed(123) #ensure reproducibility
pd.options.mode.chained_assignment = None  #hide any pandas warnings

##  **2. 데이터 불러오기**
* BostonHousing Dataset
*  보스턴 지역의 주택 가격(중앙값)을 예측하기 위한 데이터셋

In [ ]:
dt = pd.read_csv("HousingData.csv")
#dt = pd.read_csv("heart.csv")
dt.head(10)


```
  - 1. CRIM: 자치시(town) 별 1인당 범죄율     
  - 2. ZN: 25,000 평방피트를 초과하는 거주지역의 비율     
  - 3. INDUS: 비소매상업지역이 점유하고 있는 토지의 비율
  - 4. CHAS: 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1, 아니면 0)
  - 5. NOX: 10ppm 당 농축 일산화질소
  - 6. RM: 주택 1가구당 평균 방의 개수
  - 7. AGE: 1940년 이전에 건축된 소유주택의 비율
  - 8. DIS: 5개의 보스턴 직업센터까지의 접근성 지수
  - 9. RAD: 방사형 도로까지의 접근성 지수
  - 10. TAX: 10,000 달러 당 재산세율
  - 11. PTRATIO: 자치시(town)별 학생/교사 비율
  - 12. B: 1000(Bk - 0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 말함.
  - 13. LSTAT: 모집단의 하위계층의 비율(%)
  - 14.  MEDV: 본인 소유의 주택가격(중앙값) (단위: $1,000)

```

### **간단한 EDA**

- 데이터 크기 확인

In [ ]:
dt.columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX','PTRATIO', 'B', 'LSTAT', 'MEDV']

In [ ]:
print("\n데이터 크기 (행, 열):")
print(dt.shape)

- 데이터 타입 확인

In [ ]:
print("\n데이터 타입:")
print(dt.dtypes)

- 데이터 결측치 확인

In [ ]:
print(dt.isnull().sum())

- 결측치 제거

In [ ]:
dt = dt.dropna()
dt.isnull().sum()

- 데이터 중복 확인

In [ ]:
print("\n중복 데이터 개수:")
print(dt.duplicated().sum())

- 변수별 분포 확인

In [ ]:
# 변수별 히스토그램
plt.figure(figsize=(15, 15))
columns = dt.columns
for i, column in enumerate(columns):
    plt.subplot(5, 3, i + 1)
    plt.hist(dt[column], bins=10, color='skyblue', edgecolor='black')
    plt.title(column)
    plt.tight_layout()

plt.show()

- 기본 통계량 확인

In [ ]:
dt.describe()

- 상관계수 확인

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터프레임 생성 (예시)
# 예: X_test는 테스트 데이터셋
data = dt.copy()

# 상관계수 행렬 계산
correlation_matrix = data.corr()

# 상관계수 히트맵
plt.figure(figsize=(10,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

##  **3. 모델 학습**

### **3-1. 데이터셋 분할**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dt.drop('MEDV', axis=1), dt['MEDV'], test_size=0.2, random_state=10)

### **3-2. 데이터 셋의 크기 확인**

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape #데이터셋 크기 확인

### **3-3. 랜덤 포레스트 모델 생성 및 훈련**
* RandomForestRegressor:  회귀 문제를 해결하기 위해 여러 개의 결정 트리(Decision Tree)로 구성된 앙상블 모델

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=10) 
model.fit(X_train, y_train)

### **3-4. 예측 수행**

In [ ]:
y_pred = model.predict(X_test)

### **3-5. 예측 결과 확인**

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 예측값과 실제값으로 성능 평가
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 평가 결과 출력
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r2)

##  **4. 예측 결과 해석(The Explanation)**

###  **4-1. 랜덤 포레스트(RandomForest feature Importance)**

##### **문제 1. 빈칸{}을 완성하여 랜던 포레스트의 Feature Importance를 내림차순 정렬하여 BarPlot으로 시각화한 후, Feature Importance 상위 2개의 변수를 구하시오**

In [ ]:
importance = model.{} 
feature = X_train.columns 
importances = pd.DataFrame()
importances['feature'] = feature #importances 데이터프레임의 'feature' 열에 특성 이름을 저장
importances['importances'] = importance #importances 데이터프레임의 'importances' 열에 특성 중요도를 저장
importances.{}('importances', ascending=False, inplace=True) 
importances.reset_index(drop=True, inplace=True) 
importances

In [ ]:
plt.figure(figsize=(8, 4))
sns.{}
plt.title('Random Forest Feature Importances', fontsize=18)
plt.show()

###  **4-2. Permutation Importance**

##### **문제 2. 빈 칸을 완성하여 Permutation Importance를 시각화 한 후, 상위 2개의 변수를 구하시오**

In [ ]:
perm = {} #Permutation Importance 객체 생성 후 학습
eli5.{}(perm, feature_names = X_test.columns.tolist()) 

###  **4-3. PDP & ICE**

####  **4-3-(1). PDP**

##### **문제 3. CRIM과 TAX의 Partial Dependence Plot을 각각 그리고 결과를 해석하시오**

In [ ]:
fig = plt.figure(figsize=(8, 4)) 
fig.set_facecolor('white') 
ax = fig.add_subplot()
#Partial Dependence Plot 코드를 작성하시오

####  **4-3-(2). ICE**

In [ ]:
feature_names

##### **문제 4. 빈칸을 완성하여 LSTAT 변수의 값이 변화할 때, 개별 데이터가 모델에서 예측하는 결과가 어떻게 변하는지를 시각화하시오**

In [ ]:
fig = plt.figure(figsize=(8, 4))
fig.set_facecolor('white')
ax = fig.add_subplot()
{}
PartialDependenceDisplay.from_estimator(model, 
                                        X_test,
                                        {}, 
                                        feature_names={},
                                        kind='{}', 
                                        ax=ax
                                       )

####  **4-3-(3). PDP & ICE 동시에 시각화**

##### **문제 5. 빈칸을 완성하여 LSTAT 변수의 값이 변화할 때, PDP와 ICE 결과를 하나의 Plot 안에 동시에 시각화하고 결과를 해석하시오**

In [ ]:
fig = plt.figure(figsize=(8, 4))
fig.set_facecolor('white')
ax = fig.add_subplot()
PartialDependenceDisplay.from_estimator(model, 
                                        X_test, 
                                        {}, 
                                        feature_names={}, 
                                        kind={},
                                        {}={'color':'blue', 'alpha':0.5}, 
                                        {}={'color':'red', 'linewidth':2}, 
                                        ax=ax
                                       )

####  **4-3-(4). 두 변수의 조합이 모델의 예측에 어떻게 영향 시각화**

#### **문제 6.CRIM 변수와 AGE 변수의 조합의 PDF로 시각화하고 결과를 해석하시오**

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
PartialDependenceDisplay.from_estimator(estimator=model, 
                                        X=X_test, 
                                        {},
                                        feature_names={},
                                        kind={},
                                        ax=ax)

###  **4-4. SHAP**
* 각 특징이 모델의 예측에 얼마나 기여했는지를 설명
* 게임 이론에서 유래한 Shapley 값을 기반으로 하며, 모든 가능한 특징 조합에 대한 기여도를 계산하여 각 특징의 중요도를 평가

####  **4-4-(1). SHAP Summary Plot(막대 그래프(bar plot) 형식)**

#### **문제 7. 빈칸을 완성하여 Test Data의 SHAP Summary Plot을 막대 그래프(bar plot)과 점 그래프(dot plot) 형식으로 시각화하시오**

In [ ]:
explainer = {} # TreeExplainer 객체 생성
shap_values = explainer.shap_values() #SHAP값 산출
shap.summary_plot({})


####  **4-4-(2). SHAP Force Plot (단일 데이터 포인트)**
* 특정 데이터 포인트에 대해 모델이 예측한 결과를 해석

#### **문제 8. 빈칸을 완성하여 Test Data 10번째, 20번째 데이터에 대한 예측 결과를 SHAP Force Plot을 그려 해석하시오** 

In [ ]:
House = X_test.iloc[{}].astype(float) 
explainer = shap.TreeExplainer(model)
shap.initjs() 
shap_values = {} 
shap.force_plot(base_value={}, shap_values={}, features={}, feature_names={}, figsize=(2,10))

####  **4-4-(3). SHAP 의존성 플롯(dependence plot)**

##### **문제 9. 빈칸을 완성하여 Test Data 데이터에 대해 LSTAT와 RM의 변수의 상호작용이 집값에 미치는 결과를 Plot을 그려 해석하시오** 

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.decision_plot({})

####  **4-4-(4). SHAP Force Plot (다수의 데이터 포인트)**
* 특정 데이터 포인트들의 모델 예측에 대한 해석을 시각화
* 각각의 데이터 포인트가 모델 예측에 어떤 영향을 미쳤는지를 시각화

##### **문제 10. 빈칸을 완성하여 Train 데이터에 대해 50개의 데이터 예측 결과를 Force Plot을 그려 해석하시오** 

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train.iloc[50:100]) 
shap.{}({}, {}, {},figsize=(2,8))

###  **4-5. LIME**

##### **문제 11. 빈칸을 완성하여 11번째 Test 데이터에 대해 예측 결과를 LIME을 통해 해석하시오** 

In [ ]:
interpretor = lime_tabular.LimeTabularExplainer( 
    training_data=X_train.values,
    feature_names=X_train.columns,
    mode={}) 

In [ ]:
exp = interpretor.explain_instance( 
      data_row=X_test.iloc[10],
      predict_fn={} 
  )

In [ ]:
exp.as_list()

In [ ]:
exp.show_in_notebook({}) 